In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import *
from PIL import Image

from torchvision import transforms
import torchvision.transforms.functional as tf
import os
from torch.optim import RMSprop
from torchvision.utils import make_grid
from pylab import plt
import Mod.res_block as resblock

In [2]:
#参数配置
#path=os.path.abspath('../../dataset/Animals_with_Attributes2/')
tar_path=os.path.abspath('../../dataset/GZSL_for_AWA/Animals_with_Attributes2/')
bat_size=32
img_size=227
gpu=True
tr_epoch=2
worker=2


transform1=transforms.Compose([
    transforms.Resize((img_size,img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3,[0.5]*3)
])

In [3]:
#数据集载入
class SeenData(Dataset):
    def __init__(self,root,transforms=None,train_data=True,label=None):
        #路径
        #self.path=root
        
        if train_data==True:
            #root_label=os.path.abspath('../dataset/Animals_with_Attributes2')
            label_path=[x.path for x in os.scandir(root) if 'trainclass' in x.path]
            word=[]
            for i,file_label in enumerate(label_path):
                f=open(file_label,'r')
                while True:
                    temp_w=f.readline().strip()
                    if not temp_w:
                        break
                    word.append(temp_w)
                f.close()
            
            path_dir=[]
            self.label_list=word
            for key_word in self.label_list:
                [path_dir.append(x.path) for x in os.scandir(root+'/trainingset/') if x.name.endswith(key_word)]
                
            pic_path=[]
            label_t=[]
            for l,train_root in enumerate(path_dir):#对训练集文件夹遍历
                #[pic_path.append(x.path) for x in os.scandir(train_root) if x.name.endswith(".jpg")]#每个文件夹里的图片地址
                for x in os.scandir(train_root):
                    if x.name.endswith(".jpg"):
                        pic_path.append(x.path)
                        label_t.append(l)
                        
            #pic_path=path_dir#图片文件夹目录
            label_np=np.array(label_t)
            label_tensor=torch.from_numpy(label_np).type(torch.LongTensor)
            self.images_files=pic_path #图片目录
            self.transforms=transforms
            self.label=label_tensor
            
        else:
            self.path#测试机读取
            
    def __getitem__(self,index):
        try:
            pic_data=self.transforms(Image.open(self.images_files[index]))
            
        except RuntimeError:
            print(self.images_files[index])
            return self.transforms(Image.open(self.images_files[index-1])),self.label[index]
            
        else:
            return pic_data,self.label[index]
        
    def __len__(self):
        return len(self.images_files)
        

In [4]:
#网络
def Conv1(in_planes, places, stride=2):
    return nn.Sequential(
        nn.Conv2d(in_channels=in_planes,out_channels=places,kernel_size=7,stride=stride,padding=3, bias=False),
        nn.BatchNorm2d(places),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    )

class Bottleneck(nn.Module):
    def __init__(self,in_places,places, stride=1,downsampling=False, expansion = 4):
        super(Bottleneck,self).__init__()
        self.expansion = expansion
        self.downsampling = downsampling

        self.bottleneck = nn.Sequential(
            nn.Conv2d(in_channels=in_places,out_channels=places,kernel_size=1,stride=1, bias=False),
            nn.BatchNorm2d(places),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=places, out_channels=places, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(places),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=places, out_channels=places*self.expansion, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(places*self.expansion),
        )

        if self.downsampling:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels=in_places, out_channels=places*self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(places*self.expansion)
            )
        self.relu = nn.ReLU(inplace=True)
    def forward(self, x):
        residual = x
        out = self.bottleneck(x)

        if self.downsampling:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)
        return out
    
class ResNet(nn.Module):
    def __init__(self,blocks, num_classes=1000, expansion = 4):
        super(ResNet,self).__init__()
        self.expansion = expansion

        self.conv1 = Conv1(in_planes = 3, places= 64)

        self.layer1 = self.make_layer(in_places = 64, places= 64, block=blocks[0], stride=1)
        self.layer2 = self.make_layer(in_places = 256,places=128, block=blocks[1], stride=2)
        self.layer3 = self.make_layer(in_places=512,places=256, block=blocks[2], stride=2)
        self.layer4 = self.make_layer(in_places=1024,places=512, block=blocks[3], stride=2)

        #self.avgpool = nn.AvgPool2d(7, stride=1)
        self.avgpool=nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(2048,num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def make_layer(self, in_places, places, block, stride):
        layers = []
        layers.append(Bottleneck(in_places, places,stride, downsampling =True))
        for i in range(1, block):
            layers.append(Bottleneck(places*self.expansion, places))

        return nn.Sequential(*layers)


    def forward(self, x):
        x = self.conv1(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [5]:
net=ResNet([3, 4, 23, 3],num_classes=40)
loss_func=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(params=net.parameters(),lr=0.001)
indata=SeenData(tar_path,transform1)
data_training=DataLoader(indata,bat_size,shuffle = True,num_workers=worker)
#resb=resblock.ResBlock()
if gpu:
    
    net.cuda()

In [6]:
#training
Loss=[]
for epoch in range(tr_epoch):
    training_loss=0.0
    for iter_batch,batch in enumerate(data_training):
        
        data=batch[0]
        tar_label=batch[1]
        if gpu:
            data=data.cuda()
            tar_label=tar_label.cuda()
        output=net(data)
        loss=loss_func(output,tar_label)
        #print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        training_loss+= loss.item()# * batch.size(0)
        
    print(training_loss)
    Loss.append(training_loss)
        
        
    if (epoch+1)%400==0:
        torch.save(net.state_dict(),'./models/'+str(epoch+1)+'seenResNet.pkl')
Loss0 = np.array(Loss)
print(Loss0)
#torch.save(Loss0,'./models/train_loss.pth')  
np.save('./models/train_loss_resnet.npy',Loss0)

/home/zhaozhi/dataset/GZSL_for_AWA/Animals_with_Attributes2/trainingset/collie/collie_10718.jpg
2402.372113466263
/home/zhaozhi/dataset/GZSL_for_AWA/Animals_with_Attributes2/trainingset/collie/collie_10718.jpg
2076.705848813057
[2402.37211347 2076.70584881]
